<a href="https://colab.research.google.com/github/Nacho2904/orga_de_datos/blob/main/tp_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Práctico II: Spark

## Setup

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 42 kB/s 
     |████████████████████████████████| 199 kB 54.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=e8a6dfd83b5f1e0e9a04279e6814e1d43738e936ba20439006a18a5e5aa52749
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upg

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext

In [3]:
import pandas as pd
import numpy as np
from google.colab import drive 

drive.mount('/content/gdrive')
path_a_cast = 'gdrive/MyDrive/Parquets/cast.parquet'
path_a_companies = 'gdrive/MyDrive/Parquets/companies.parquet'
path_a_crew = 'gdrive/MyDrive/Parquets/crew.parquet'
path_a_actores = 'gdrive/MyDrive/Parquets/imdb_actors.parquet'
path_a_keywords = 'gdrive/MyDrive/Parquets/keywords.parquet'
path_a_links = 'gdrive/MyDrive/Parquets/links.parquet'
path_a_movies = 'gdrive/MyDrive/Parquets/movies.parquet'
path_a_ratings = 'gdrive/MyDrive/Parquets/ratings.parquet'


Mounted at /content/gdrive


In [4]:
spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


## S7

¿Cuál fue el año más mortal para los actores y actrices?

In [ ]:
AÑO_FALLECIMIENTO = 3
CONTEO_MUERTES = 1
rdd_actores = sqlContext.read.parquet(path_a_actores).rdd

In [ ]:
rdd_años_de_fallecimiento = rdd_actores.map(lambda row: row[AÑO_FALLECIMIENTO])
rdd_años_de_fallecimiento.first()

'1987'

In [ ]:
rdd_años_de_fallecimiento = rdd_años_de_fallecimiento.filter(str.isnumeric)
rdd_años_de_fallecimiento = rdd_años_de_fallecimiento.map(int)
dict_con_conteo = dict(rdd_años_de_fallecimiento.countByValue())
año_mas_mortal = sorted([[llave, dict_con_conteo[llave]] for llave in dict_con_conteo], 
                        key = lambda año_y_conteo: año_y_conteo[CONTEO_MUERTES])[-1]
print(año_mas_mortal)

[2021, 3409]


## S23

¿Cuál es el usuario más pesimista (menor rating promedio) para cada género de películas? Considere solo usuarios con más de 20 reviews

### Resolución

In [6]:
rdd_ratings = sqlContext.read.parquet(path_a_ratings).rdd
rdd_movies = sqlContext.read.parquet(path_a_movies).rdd

Primero filtraremos los usuarios por aquellos que tengan más de 20 reviews

In [10]:
rdd_ratings_as_pairs = rdd_ratings.map(lambda row: (row[0], (row[1], row[2])))
count_of_ratings_per_user = rdd_ratings_as_pairs.countByKey()
users_with_plus_20_ratings = {user for user in count_of_ratings_per_user if count_of_ratings_per_user[user] > 20}
len(users_with_plus_20_ratings)

161497

In [16]:
rdd_ratings_users_with_more_than_20_reviews = rdd_ratings_as_pairs.filter(lambda row: row[0] in users_with_plus_20_ratings)
rdd_ratings_with_movie_id_as_key = rdd_ratings_users_with_more_than_20_reviews.map(lambda row: (str(row[1][0]), (row[0], row[1][1])))

In [25]:
rdd_movies_and_genres_as_pairs = rdd_movies.map(lambda row: (row[5], set(row[3].split(',') if row[3] else {})))
rdd_filtered_ratings_with_genres = rdd_ratings_with_movie_id_as_key.join(rdd_movies_and_genres_as_pairs)

In [26]:
rdd_filtered_ratings_with_genres_as_user_pairs = rdd_filtered_ratings_with_genres.map(lambda row: (row[1][0][0], (row[1][0][1], row[1][1])))

In [32]:
rdd_filtered_ratings_with_genres_as_user_pairs.first()

(229, (3.0, {'Crime', 'Drama', 'Mystery', 'Thriller'}))

In [27]:
genres = rdd_movies_and_genres_as_pairs.map(lambda row: row[1]).reduce(
    lambda genres_1, genres_2: genres_1.union(genres_2))

In [31]:
genres_with_sum_of_ratings_and_counts = {genre: [0, 0] for genre in genres}

In [52]:
def create_dict_for_rating(rating):
  rating_given, genres_of_movie_rated = 0, 1
  genres_with_sum_of_ratings_and_counts = {genre: [0, 0] for genre in genres}
  
  for genre in rating[genres_of_movie_rated]:
    genres_with_sum_of_ratings_and_counts[genre][0] += rating[rating_given]
    genres_with_sum_of_ratings_and_counts[genre][1] += 1
  
  return genres_with_sum_of_ratings_and_counts  

def merge_rating_dicts(rating_dict_1, rating_dict_2):
  return {genre: [rating_dict_1[genre][0] + rating_dict_2[genre][0],
                  rating_dict_1[genre][1] + rating_dict_2[genre][1]] for genre in genres}

rdd_user_mean_ratings_per_genre = rdd_filtered_ratings_with_genres_as_user_pairs.map(
  lambda row: (row[0], create_dict_for_rating(row[1]))).reduceByKey(merge_rating_dicts).map(
      lambda row: (row[0], {genre: row[1][genre][0]/row[1][genre][1] if row[1][genre][1] else np.Inf for genre in genres}) 
  )

In [53]:
rdd_user_mean_ratings_per_genre.first()

(1060,
 {'Adventure': 3.78125,
  'Romance': 3.8846153846153846,
  'Foreign': inf,
  'Science Fiction': 3.6153846153846154,
  'Drama': 4.230769230769231,
  'Animation': 4.5,
  'Western': 4.0,
  'Documentary': 3.25,
  'Thriller': 4.293103448275862,
  'Fantasy': 4.1,
  'War': 5.0,
  'Horror': 4.0,
  'Action': 4.117647058823529,
  'History': 4.5,
  'Crime': 4.205882352941177,
  'Comedy': 4.092592592592593,
  'Mystery': 3.8076923076923075,
  'Family': 4.1875,
  'TV Movie': inf,
  'Music': 4.25})

In [56]:
most_pessimistic_users_by_genre = {genre: rdd_user_mean_ratings_per_genre.min(key = lambda row: row[1][genre])
                                   for genre in genres}

In [58]:
most_pessimistic_users_by_genre_nicely_formatted = {genre: (most_pessimistic_users_by_genre[genre][0], 
                                                            most_pessimistic_users_by_genre[genre][1][genre])
                                                            for genre in genres}

most_pessimistic_users_by_genre_nicely_formatted

{'War': (177248, 0.5),
 'Foreign': (267084, 0.5),
 'Romance': (111772, 0.5),
 'Comedy': (700, 0.5),
 'Action': (111772, 0.5),
 'Mystery': (111772, 0.5),
 'Family': (32560, 0.5),
 'Crime': (65404, 0.5),
 'TV Movie': (33420, 0.5),
 'Documentary': (122220, 0.5),
 'Science Fiction': (127668, 0.5),
 'Drama': (111772, 0.5),
 'Horror': (6900, 0.5),
 'Music': (44584, 0.5),
 'History': (132276, 0.5),
 'Fantasy': (168084, 0.5),
 'Adventure': (700, 0.5),
 'Western': (88588, 0.5),
 'Animation': (214972, 0.5),
 'Thriller': (127668, 0.5)}

## S25

¿Cuál es el percentil 95 de la popularidad?

In [ ]:
POPULARITY = 10
rdd_movies = sqlContext.read.parquet(path_a_movies).rdd

rdd_popularities = rdd_movies.map(lambda row: row[POPULARITY]).filter(lambda row: row)
number_of_movies = rdd_popularities.count()
top_5_per_cent = rdd_popularities.takeOrdered(int(number_of_movies*0.05), key = lambda x: -x)
percentile_95th = top_5_per_cent[-1]
print(percentile_95th)

11.063822


## S11

 Según los cinéfilos, ¿cuál es el rating promedio de las 5 películas más populares? Un usuario es cinéfilo cuando puntuó más de 50 películas y todas las puntuaciones son de más de 2.5 estrellas. Indicar id, título, popularidad y rating promedio de la película.

In [ ]:
rdd_ratings = sqlContext.read.parquet(path_a_ratings).rdd
rdd_movies = sqlContext.read.parquet(path_a_movies).rdd

En primer lugar reordenamos el rdd para que cada fila sea un par clave-valor, con las claves siendo los userId y los valores siendo los ratings de las pelis y sus respectos id

In [ ]:
rdd_ratings_key_value = rdd_ratings.map(lambda row: (row[0], [row[1], row[2]]))

Ahora filtramos el rdd de ratings. Primero agrupamos todos los ratings por id de usuario, y filtramos los usuarios que no tengan más de 50 reviews, y que tengan una review con rating menor a 2.5, es decir, que no son considerados cinéfilos.

In [ ]:
user_has_no_ratings_below = lambda user, threshold: all([rating[1] > threshold for rating in user[1]]) 
user_has_more_than_25_ratings = lambda user: len(user[1]) > 50
rdd_ratings_grouped_per_user = rdd_ratings_key_value.groupByKey().map(lambda row: (row[0], list(row[1])))
rdd_usuarios_cinefilos = rdd_ratings_grouped_per_user.filter(lambda user: user_has_no_ratings_below(user, 2.5) and 
                                                         user_has_more_than_25_ratings(user)).map(lambda user: user[0])

In [ ]:
set_usuarios_cinefilos = set(rdd_usuarios_cinefilos.collect())
rdd_ratings_key_value_cinefilos = rdd_ratings_key_value.filter(lambda row: row[0] in set_usuarios_cinefilos)

Joineamos las tablas de ratings y pelis para obtener solo los ratings de pelis que los cinéfilos han rateado, y para poder tener en cuenta las popularidades

In [ ]:
rdd_ratings_cinephiles_movies_as_keys = rdd_ratings_key_value_cinefilos.map(lambda row: (str(row[1][0]), row[1][1]))
rdd_movies_as_pairs = rdd_movies.map(lambda row: (str(row[5]), row[10]))
rdd_rating_cinephiles_with_popularities = rdd_ratings_cinephiles_movies_as_keys.join(rdd_movies_as_pairs)


- El primer map reordena el rdd en pares clave-valor en el que las claves son los id de las pelis y los valores son el rating y la popularidad
- El reduceByKey arma un nuevo rdd con la sumatoria de ratings, count y popularidad por cada peli
- El map final obtiene un rdd con el promedio de ratings y popularidad por peli

In [ ]:
rdd_cinephiles_mean_ratings_for_each_movie = rdd_rating_cinephiles_with_popularities.map(
    lambda row: (row[0], (row[1][0], row[1][1], 1))).reduceByKey(
      lambda value_1, value_2: (value_1[0] + value_2[0], value_1[1], value_1[2] + value_2[2])).map(
        lambda row: (row[0], (row[1][0]/row[1][2], row[1][1])))

Finalmente, obtenemos las cinco pelis con mayor popularidad de las que tienen ratings de cinefilos, junto con su rating promedio y su id

In [ ]:
rdd_cinephiles_mean_ratings_for_each_movie.takeOrdered(5, lambda row: -row[1][1])

[('680', (4.071428571428571, 140.950236)),
 ('155', (4.0, 123.167259)),
 ('78', (3.5833333333333335, 96.272374)),
 ('550', (3.660377358490566, 63.869599)),
 ('278', (3.375, 51.645403))]

## S34

 ¿Cuál es la probabilidad de que una película de un género en particular tenga una calificación promedio mayor a 3 (utilizando la tabla de ratings)? Obtenga las probabilidades de cada género posible.

In [ ]:
rdd_ratings = sqlContext.read.parquet(path_a_ratings).rdd
rdd_movies = sqlContext.read.parquet(path_a_movies).rdd

In [ ]:
rdd_movies_id_and_genre = rdd_movies.map(lambda row: (row[5], set(row[3].split(',') if row[3] else {})))

genres = rdd_movies_id_and_genre.map(lambda row: row[1]).reduce(
    lambda genres_1, genres_2: genres_1.union(genres_2))

In [ ]:
rdd_ratings_filtered = rdd_ratings.map(lambda row: (str(row[1]), row[2]))
rdd_mean_ratings = rdd_ratings_filtered.groupByKey().map(lambda row: (row[0], np.sum(np.array(list(row[1])))/len(list(row[1]))))
rdd_mean_ratings_and_genres = rdd_mean_ratings.join(rdd_movies_id_and_genre).map(lambda row: (row[1][1], row[1][0]))

[('3061', 3.8211314475873546),
 ('4368', 2.428208386277001),
 ('60758', 3.409340659340659),
 ('3807', 3.159375),
 ('157967', 3.6666666666666665)]

In [ ]:
get_count_of_movies_of_genre = lambda genre: rdd_mean_ratings_and_genres.filter(lambda row: genre in row[0]).count()
get_count_of_good_movies_of_genre = lambda genre: rdd_mean_ratings_and_genres.filter(lambda row: genre in row[0] and row[1] > 3).count()
get_probability_of_good_movie_of_genre = lambda genre: get_count_of_good_movies_of_genre(genre)/get_count_of_movies_of_genre(genre)

genre_with_probabilities = {genre: get_probability_of_good_movie_of_genre(genre) for genre in genres}
genre_with_probabilities

{'Romance': 0.6459369817578773,
 'Horror': 0.6251728907330567,
 'Music': 0.657258064516129,
 'TV Movie': 0.6551724137931034,
 'Drama': 0.6382921589688507,
 'Family': 0.6606217616580311,
 'Action': 0.64,
 'War': 0.6147186147186147,
 'Adventure': 0.6619519094766619,
 'Science Fiction': 0.6441821247892074,
 'Comedy': 0.6393752957879791,
 'History': 0.6143790849673203,
 'Mystery': 0.6357894736842106,
 'Animation': 0.6860986547085202,
 'Western': 0.6702702702702703,
 'Foreign': 0.6125461254612546,
 'Documentary': 0.5964214711729622,
 'Thriller': 0.6370007007708479,
 'Crime': 0.6515986769570011,
 'Fantasy': 0.6285097192224622}